
**Data Dazzlers✨**


In [1]:
pip install PyPDF2


In [2]:
pip install python-pptx


In [3]:
pip show python-pptx


Name: python-pptx
Version: 0.6.23
Summary: Generate and manipulate Open XML PowerPoint (.pptx) files
Home-page: https://github.com/scanny/python-pptx
Author: Steve Canny
Author-email: python-pptx@googlegroups.com
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: lxml, Pillow, XlsxWriter
Required-by: textract


In [4]:
pip install pytesseract

In [5]:
pip install python-docx

In [6]:
pip install textract


In [7]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [8]:
import os
import PyPDF2
import pptx
from bs4 import BeautifulSoup
import pytesseract
from PIL import Image
import textract
from docx import Document

In [9]:
def extract_text_from_docx(file_path):
    doc = Document(file_path)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return '\n'.join(full_text)


def extract_text_from_pdf(file_path):
     text = ""
    with open(file_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()
    return text

def extract_text_from_ppt(file_path):
    text = ""
    ppt = python_pptx.Presentation(file_path)
    for slide in ppt.slides:
        for shape in slide.shapes:
            if hasattr(shape, "text"):
                text += shape.text + "\n"
    return text

def extract_text_from_html(file_path):
    with open(file_path, 'r') as file:
        html_content = file.read()
        soup = BeautifulSoup(html_content, 'html.parser')
        return soup.get_text()

def extract_text_from_image(file_path):
    image = Image.open(file_path)
    text = pytesseract.image_to_string(image)
    return text

# def extract_text_from_doc(file_path):
#     doc = docx.Document(file_path)
#     text = ""
#     for paragraph in doc.paragraphs:
#         text += paragraph.text + "\n"
#     return text
def extract_text_from_doc(file_path):
    text = textract.process(file_path).decode("utf-8")
    return text

def extract_text_from_file(file_path):
    _, file_extension = os.path.splitext(file_path)
    if file_extension.lower() == '.pdf':
        return extract_text_from_pdf(file_path)
    elif file_extension.lower() == '.pptx':
        return extract_text_from_ppt(file_path)
    elif file_extension.lower() == '.html':
        return extract_text_from_html(file_path)
    elif file_extension.lower() in ['.png', '.jpg', '.jpeg']:
        return extract_text_from_image(file_path)
    elif file_extension.lower() == '.docx':
        return extract_text_from_docx(file_path)
    # elif file_extension.lower() == '.doc':
    #     return extract_text_from_docx(file_path)
    else:
        return "Unsupported file format"

In [10]:
import re
import os

def list_files_in_directory(directory):
    files = os.listdir(directory)
    return files


def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text)
    return text

def identify_sections(resume_text):
    # Preprocess the resume text
    resume_text = preprocess_text(resume_text)

    # Define regular expressions for common section headers
    section_headers_regex = {
        'education': r'\b(education|academic qualifications)\b',
        'experience': r'\b(experience|work experience|professional experience)\b',
        'skills': r'\b(skills|technical skills)\b',
        'projects': r'\b(projects|personal projects|side projects)\b',
        # Add more section headers as needed
    }

    sections = {}

    # Iterate over each section header regex
    for section, regex_pattern in section_headers_regex.items():
        # Search for the section header in the preprocessed resume text
        match = re.search(regex_pattern, resume_text, re.IGNORECASE)
        if match:
            # Extract the section content starting from the position of the match
            section_start_index = match.end()
            # Find the end of the section or the start of the next section
            next_section_start = len(resume_text)
            for next_section in section_headers_regex.values():
                next_match = re.search(next_section, resume_text[section_start_index:], re.IGNORECASE)
                if next_match:
                    next_section_start = section_start_index + next_match.start()
                    break
            # Extract the section content
            section_content = resume_text[section_start_index:next_section_start].strip()
            sections[section] = section_content

    return sections

In [11]:
def section_bifercation():
  directory_path = '/content/drive/MyDrive/Colab Notebooks/resume_parser_test'  # Replace with your directory path
  files = list_files_in_directory(directory_path)
  experience_list=[]
  for file in files:
    # resume_text =extract_text_from_file('/content/drive/MyDrive/resume_parser/Charles Obuseh.pdf')
    input_file_path='/content/drive/MyDrive/Colab Notebooks/resume_parser_test/'+str(file)
    resume_text =extract_text_from_file(input_file_path)
    # Identify sections in the resume
    resume_sections = identify_sections(resume_text)
    for section, content in resume_sections.items():
        if section.upper()=='EXPERIENCE':
          experience_list.append(content)
        print(f"--- {section.upper()} ---")
        print(content)
        print()
    print('*********************************************************************************')


In [12]:
def extract_keywords(text):
    # Define keywords or key phrases relevant to each section
    skills_keywords = [
        "Python", "SQL", "Bash", "Java", "Scala", "HTML", "CSS", "JavaScript", "PL/SQL", "MATLAB", "R",
        "Flask", "Git", "Pandas", "Tableau BI", "ETL", "Apache Spark", "Hadoop", "HDFS", "Snowflake Data Factory",
        "SnowSQL", "Snowpipe", "Amazon Kinesis", "Firehose", "Databricks", "AWS", "EMR", "Athena", "Lambda",
        "Amazon MW", "Apache Airflow", "AWS EC2", "QuickSight", "SNS", "Cloudera infrastructure", "Ambari",
        "Hortonworks", "Sandbox", "Zeppelin Notebook", "RDS", "MySQL", "MongoDB", "DynamoDB", "SQLAlchemy",
        "PostgreSQL", "S3 Buckets", "SQLite", "Azure", "Oracle", "Redshift", "ECS", "Docker", "Jenkins",
        "Ubuntu", "CloudFormation", "CircleCI", "Microsoft Azure", "PowerApps", "Power BI", "Jupyter Hub",
        "Analytic Development", "R-programming", "C#", "XML", "SAP Analytics Looker", "Google Charts", "Qlik",
        "SSRS", "SSIS", "Informatica", "SAC", "JIRA", "Azure DevOps", "SharePoint", "MS-Teams", "Text comprehension",
        "NLP", "Classification", "Pattern Recognition", "Imaging Recognition", "Detection", "Forecasting", "Clustering",
        "Sentiment Analysis", "Predictive Analytics", "Decision Analytics", "Association Analysis", "Attribution modeling",
        "Classification Regression Trees (CART)", "SVM", "Random Forest", "GBM", "PCA", "RNN", "Naïve Bayes",
        "Bayesian Analysis", "Statistical Inference", "Predictive Modeling", "Stochastic Modeling", "Linear Modeling",
        "Behavioral Modeling", "NumPy", "Scikit-learn", "TensorFlow", "SciPy", "Matplotlib", "Seaborn", "Keras", "NLTK",
        "Beautiful Soup", "StatsModels", "PyTorch", "OpenCV", "Sweetviz", "Theano", "Bokeh", "Plotly", "Genism", "Scrapy",
        "kivy", "Caffe2", "Spyder", "RStudio", "Visual Studio", "A/B testing", "GitHub", "Agile methodologies", "Snowflake",
        "MS Access", "Amazon S3", "Mongo DB", "My SQL", "Pyspark", "Communication", "Presentation skills", "Teamwork",
        "Leadership", "Critical thinking", "Creativity", "Problem-solving", "Accountability", "Shell Scripting", "Storm",
        "JSP", "Servlets", "Kimball data warehousing methodology", "Linear Regression", "Ridge Regression", "Polynomial Regression",
        "Lasso Regression", "Elastic Net", "k-Means", "Hierarchical Clustering", "Latent Dirichlet Allocation (LDA)",
        "Amazon Web Services", "GIT", "SVN", "CVS", "C++", "Arduino", "Libraries", "Dask", "Postgres", "Geopy",
        "Geopandas", "Algorithms", "Deep Learning", "Time Series", "Mathematical Core", "Statistics",
        "Probability", "Multivariate Calculus", "Optimization", "Functional", "Machine Learning", "Natural Language Processing (NLP)",
        "Artificial Intelligence", "Business", "Collaborative", "Attention to Detail", "Enterprise UAV Systems Integration",
        "Developing", "Distributed Enterprise-GIS Systems (ESRI, Open Source)", "ArcGIS Enterprise", "GeoEvent Server",
        "ArcGIS ImageServer", "ArcGIS Notebook Server", "Geoserver", "Mapserver", "Linux SysAdmin", "Computer Vision",
        "Pose Estimation", "Structure-from-Motion", "GIS 3D Reconstructions", "MVS", "Poisson Surface Reconstruction",
        "Pipelines", "Geometric Deep Learning", "Point Clouds", "PointNet", "ShapeNet", "SyncSpecCNN", "YOLO3",
        "Nvidia NERF implementations", "UAVs", "Databases", "SAP HANA", "PostGreSQL", "PostGIS", "Cassandra",
        "HA Virtualization", "Hyper-V", "Xen", "ESXi", "AWS/Azure", "Containerization", "LXC", "Kubernetes",
        "ProxMoxVE", "SSMS", "Sage", "IoT", "Virtual Machines", "GCP", "BigTable", "DataLab"
    ]
    experience_keywords = [
        "Customer Solutions Data Engineer", "Systems and Data Engineer", "Consultant Measurements Field Engineer",
        "Senior Technical Account Manager", "Engineering Coordinator", "COGNITIVE SCIENCE INSTITUTE Lab Manager",
        "Postdoctoral Research Fellow", "Graduate Research Fellow", "Data Engineer KPMG, Tampa FL",
        "Sr. Data Engineer Walmart, Sunnyvale CA", "Python Developer, Charter, MO", "Data Engineer ViacomCBS, NYC",
        "Big Data Engineer WebSoc Technologies, Hyderabad India", "Data Engineer ARISSOFT SOLUTIONS, Hyderabad, India",
        "Senior Data Engineer, Dhruvsoft Services Private Limited, Hyderabad, India", "Data Analyst, AI/ML Intern, Modak Analytics LLP, India",
        "Data Engineer, Live Life Love Life Charity Foundation- Fundraising Coordinator, Vistara",
        "ETL Developer, Charles Schwab", "Allstate, Associate Data Scientist Bangalore, INDIA Datakalp",
        "Research Data Engineerin Intern Pittsburgh, USA Robert Bosch R&D", "Teaching Fellow New York City, New York University - Leonard N. Stern School of Business",
        "New York", "Software Engineer Dubai, UAE Varal Group DMCC", "Data Engineer | TC Energy | Houston, TX",
        "Odfjell Terminal | Seabrook, TX", "Enterprise, Products (Through CF) | Houston, TX", "Analytics Developer",
        "Undergraduate Researcher", "Senior Data Analyst Tesoro Logistics (Through IG) | San Antonio, TX", "Business Intelligence Analyst",
        "Systems Administrator", "Research Assistant", "Big Data Engineer", "Project Lead", "Data Scientist",
        "Data Science Intern", "Data Analysis Research Associate", "Assistant System Administrator", "Freelance Developer",
        "Note Taker", "Financial Systems and Data Intern", "Senior Analyst", "Senior Associate - Data Science & Quants Lab",
        "Associate Software Engineer - Data Science & Quants Lab", "Clark Associates Inc Financial Systems and Data Intern",
        "Dacapo Brokerage India Private Limited Senior Analyst", "Onsite Supply Chain Analytics Project - Doha, Qatar",
        "DronaMaps Private Limited Founder & CEO / August 2016 to July 2022", "Sr. Database Developer", "Database and Business Intelligence Developer",
        "HITACHI ENERGY Data Science Intern", "ENERGY INFORMATICS GROUP", "NATIONAL CENTRE IN BIG DATA & CLOUD COMPUTING Data Analysis Research Associate",
        "Disaster Tweet Prediction", "Predict Churning Customers", "Climate Change Tracker"
    ]

    # Find keywords in the text
    keywords = {'skills': [], 'experience': [], 'projects': []}
    for section, keywords_list in zip(keywords.keys(), [skills_keywords, skills_keywords, skills_keywords]):
        for keyword in keywords_list:
            if keyword.lower() in text:
                keywords[section].append(keyword)

    return keywords



In [13]:
# def keyword_in_file():
#     directory_path = '/content/drive/MyDrive/Colab Notebooks/resume_parser_test'  # Replace with your directory path
#     files = list_files_in_directory(directory_path)

#     for file in files:
#         input_file_path = os.path.join(directory_path, file)
#         resume_text = extract_text_from_file(input_file_path)
#         resume_sections = identify_sections(resume_text)
#         for section, content in resume_sections.items():
#             keywords = extract_keywords(content)
#             with open(f'{section}_keywords.txt', 'a') as f:
#                 f.write(f'Resume: {file}\n')
#                 section_keywords = keywords.get(section, [])  # Get keywords or an empty list if section is not found
#                 f.write('\n'.join(section_keywords) + '\n')
#                 f.write('--------------------\n')


import os

def keyword_in_file():
    directory_path = '/content/drive/MyDrive/Colab Notebooks/resume_parser_test'  # Replace with your directory path
    files = list_files_in_directory(directory_path)

    for file in files:
        input_file_path = os.path.join(directory_path, file)
        resume_text = extract_text_from_file(input_file_path)
        resume_sections = identify_sections(resume_text)
        for section, content in resume_sections.items():
            keywords = extract_keywords(content)
            with open(f'{section}_keywords.txt', 'a') as f:
                f.write(f'Resume: {file}\n')
                section_keywords = keywords.get(section, [])  # Get keywords or an empty list if section is not found
                f.write('\n'.join(section_keywords) + '\n')
                f.write('--------------------\n')

                # Display output on the screen
                print(f'Resume: {file}')
                print(f'--- {section.upper()} ---')
                print('\n'.join(section_keywords))
                print('--------------------')

keyword_in_file()


Resume: Charles Obuseh.pdf
--- EDUCATION ---

--------------------
Resume: Charles Obuseh.pdf
--- EXPERIENCE ---
Python
SQL
R
ETL
Amazon Kinesis
Firehose
Databricks
AWS
EMR
Lambda
Apache Airflow
AWS EC2
Cloudera infrastructure
Ambari
Hortonworks
Sandbox
Zeppelin Notebook
Oracle
Snowflake
Pyspark
--------------------
Resume: Charles Obuseh.pdf
--- SKILLS ---
Python
SQL
Bash
R
Git
Pandas
Tableau BI
ETL
Apache Spark
Hadoop
HDFS
Snowflake Data Factory
SnowSQL
Snowpipe
Amazon Kinesis
Firehose
Databricks
AWS
EMR
Athena
Lambda
Amazon MW
Apache Airflow
AWS EC2
QuickSight
SNS
Cloudera infrastructure
Ambari
Hortonworks
Sandbox
Zeppelin Notebook
RDS
MySQL
MongoDB
DynamoDB
SQLAlchemy
PostgreSQL
S3 Buckets
SQLite
Azure
Oracle
Snowflake
Pyspark
GIT
Postgres
Databases
PostGreSQL
--------------------


In [14]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [15]:
pip install find-job-titles


In [16]:
from find_job_titles import Finder
from collections import defaultdict

list_of_job=defaultdict()
def find_jobs():

  finder = Finder()

  directory_path = '/content/drive/MyDrive/Colab Notebooks/resume_parser_test'  # Replace with your directory path
  files = list_files_in_directory(directory_path)
  print("Files in directory:")
  for file in files:
      print(file)
      input_file_path='/content/drive/MyDrive/Colab Notebooks/resume_parser_test/'+str(file)
      text=extract_text_from_file(input_file_path)
      titles=finder.findall(text)
      # print("Job titles found:",titles)
      matched_strings = [match.match for match in titles]
      list_of_job[file]=matched_strings
  #print(list_of_job)
  return list_of_job

In [17]:
def pre_mapping():
  f = open('/content/drive/MyDrive/Colab Notebooks/Onet.txt', 'r')
  data = f.read()
  # print(data)

  codes = []
  job_titles = []

  # Split the data into lines
  lines = data.strip().split('\n')

  # Extracting codes and job titles and appending them to respective lists
  for line in lines:
      # Splitting each line by whitespace to handle different delimiters
      parts = line.split()
      if len(parts) >= 2:
          code = parts[0]
          title = ' '.join(parts[1:])
          codes.append(code)
          job_titles.append(title)

  # Creating a list of dictionaries
  occupations_list = [{"number": code, "title": title} for code, title in zip(codes, job_titles)]

  # Printing the list of dictionaries
  #print(occupations_list)
  f.close()
  return codes,job_titles

In [18]:
import re
from gensim.models import Word2Vec
from numpy import dot
from numpy.linalg import norm

In [19]:
# Function to calculate cosine similarity
def cosine_similarity(vec1, vec2):
    return dot(vec1, vec2) / (norm(vec1) * norm(vec2))

# Assuming get_average_vector function exists
def get_average_vector(title, model):
    vectors = [model.wv[word] for word in title if word in model.wv]
    if vectors:
        return sum(vectors) / len(vectors)
    else:
        return None


In [20]:
def job_mapping(job_titles,list_of_job):
  # Preprocess job titles
  job_titles_processed = [re.sub(r'[^\w\s]', '', title).lower().split() for title in job_titles]

  mapped_jobs=[]

  for word in list_of_job.values():
      found = word
      mapped = []
      # Preprocess found values
      found_processed = [re.sub(r'[^\w\s]', '', title).lower().split() for title in found]

      # Train Word2Vec model
      model = Word2Vec(sentences=job_titles_processed, vector_size=100, window=5, min_count=1, workers=4)

      mapped_titles = set()  # Keep track of mapped titles to ensure uniqueness
      similarity_threshold = 0.5  # Set similarity threshold

      for title in found_processed:
          max_similarity = -1
          most_similar_title = None
          title_vector = get_average_vector(title, model)
          if title_vector is not None:
              for job_title in job_titles_processed:
                  job_title_vector = get_average_vector(job_title, model)
                  if job_title_vector is not None:
                      similarity = cosine_similarity(title_vector, job_title_vector)
                      if similarity > max_similarity:
                          max_similarity = similarity
                          most_similar_title = job_title
              if max_similarity >= similarity_threshold:
                  mapped.append(' '.join(most_similar_title))
                  mapped_titles.add(tuple(most_similar_title))  # Convert list to tuple
              else:
                  mapped.append(' '.join(title))  # Retrieve the original title
          else:
              mapped.append(' '.join(title))  # Retrieve the original title

  #    print(mapped)
      mapped_jobs.append(mapped)
  i=0
  for word in list_of_job.values():
    print("Extracted job roles:",word)
    print("Mapped job roles:",mapped_jobs[i])
    i+=1
    print("---------------------------------------------------------------------------")

In [21]:
section_bifercation()
keyword_in_file()
list_of_jobs = find_jobs()
codes,job_titles = pre_mapping()
job_mapping(job_titles,list_of_jobs)

--- EDUCATION ---
& certifications university of north texas | master of science | mechanical and energy engineering | denton, tx rice university | certificate | data analytics | houston, tx license licensed texas professional engineer | texas board of professional engineer and land surveyors additional

--- EXPERIENCE ---
data engineer | tc energy | houston, tx | 09/2019 – present | odfjell terminal | seabrook , tx | 02/2019 – 09/2019 | enterprise products (through cf) | houston , tx | 10/2018 – 02/2019 • design and implementation of process data pipeline for asset data migration into aws cloud environment • built snowflake data pipeline using amazon kinesis firehose starting from the aws ec2 logs to storage in snowflake and aws s3 bucket post -transformation and orchestrating through amazon managed workflows for apache airflow (mwaa) dags .. programming language used are pyspark, python, sparksql • architecting data pipeline with cloudera infrastructure – ambari, hortonworks sandbox 